# **GemmaX ChatCore**






A multimodal AI chat application built using Gemma 3 models, supporting
cloud inference, vision analysis, local JAX inference, multi-turn chat, and function calling.

**1. Setup & Installation**

In [ ]:
!pip install google-genai==1.45.0

**2. Initialize Google GenAI Client (Cloud Inference)**

In [ ]:
from google.colab import userdata
from IPython.display import HTML, Markdown, display

In [ ]:
from google import genai
from google.genai.types import Part

client = genai.Client(api_key=userdata.get('gen-ai-key'))

MODEL_ID = "models/gemma-3-27b-it"

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=["Who is Kalpana Chawla?"],
)

display(Markdown(response.text))

**3. Vision + Text (Multimodal Inference Using Gemma)**

In [ ]:
from IPython.display import Image
Image('/content/Kalpana Chawla.jpg', width=400, height=400)

In [ ]:
with open("/content/Kalpana Chawla.jpg", "rb") as f:
    image = f.read()


response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "What is this image about?",
    ],
)

display(Markdown(response.text))

In [ ]:
Image('/content/Temple.jpg',width = 400, height = 400)

In [ ]:
import time
with open("/content/unnamed.jpg", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "What is this image about and where is this place located? Tell something about the timing of this photo",
    ],
)
display(Markdown(response.text))

**4. Setup Local Gemma (JAX Backend)**

In [ ]:
import kagglehub
import os

kagglehub.login()



In [ ]:
%%capture

!pip uninstall -y -q keras-hub
!pip install -q -U keras-hub
!pip install  -q -U keras

**4.1. Configure JAX GPU Backend**

In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["KERAS_BACKEND"] = "jax"

In [ ]:
!nvidia-smi

**5. Import Dependencies**

In [ ]:
import io
import re

import keras
import keras_hub

import numpy as np
import PIL
import requests

from IPython.display import display, Markdown

**6. Utility Function for Formatting Model Output**

In [ ]:
def display_formatted_output(raw_text, only_model_result=False):

    separator = '<end_of_turn>\n<start_of_turn>model'

    if separator not in raw_text:
        print("Warning: Input text does not contain the expected separator.")
        print("Displaying raw text without formatting:\n---")
        print(raw_text)
        print("---")
        return

    parts = raw_text.split(separator, 1)

    if len(parts) != 2:
         print(f"Warning: Could not properly split the text with separator '{separator}'.")
         print("Displaying raw text without formatting:\n---")
         print(raw_text)
         print("---")
         return

    user_prompt = parts[0].replace('<start_of_turn>user', '').strip()

    model_response_raw = parts[1].replace('<end_of_turn>', '').strip()

    model_response_formatted = re.sub(r'(?<!\n)\n(?!\n)', r'<br>\n', model_response_raw)

    formatted_markdown = f"""**User:**

> {user_prompt}

---

**Model:**

{model_response_formatted}
"""
    if only_model_result:
        return display(Markdown(model_response_formatted))
    else:
      display(Markdown(formatted_markdown))

**7. Load Gemma Model Locally**

In [ ]:
gemma_lm = keras_hub.models.Gemma3CausalLM.from_preset(
    "gemma3_instruct_4b",
    dtype="bfloat16",
)

gemma_lm.summary()

In [ ]:
gemma_lm.preprocessor.max_images_per_prompt = 2
gemma_lm.preprocessor.sequence_length = 768

In [ ]:
PROMPT_TEMPLATE = """<start_of_turn>user
{question}
<end_of_turn>
<start_of_turn>model
"""

**9. Multi-turn Chat Application**



In [ ]:
def multi_turn_conversation(gemma_lm, initial_prompt):
    conversation_history = initial_prompt
    while True:
        output = gemma_lm.generate(conversation_history)
        display_formatted_output(output)

        next_prompt = input("Your next question/input (or type 'exit' to end): ")
        if next_prompt.lower() == "exit":
            break

        conversation_history += f"\n<end_of_turn>\n<start_of_turn>user\n{next_prompt}\n<end_of_turn>\n<start_of_turn>model\n"

initial_prompt = """<start_of_turn>user
Hey, what's up.
<end_of_turn>
<start_of_turn>model
"""

multi_turn_conversation(gemma_lm, initial_prompt)


In [ ]:
multi_turn_conversation(gemma_lm, initial_prompt)

**10. Function Calling Setup**

In [ ]:
!pip install CurrencyConverter
from currency_converter import CurrencyConverter
import datetime
cc_object = CurrencyConverter()

In [ ]:
def get_current_datetime():
  now = datetime.datetime.now()
  current_date = now.strftime("%Y-%m-%d")
  current_time = now.strftime("%H:%M:%S")
  return f"Current Date: {current_date}, Current Time: {current_time}"


def currency_converter(amount, source_currency, destinatoon_currency):
  print(f"(py function) fetching the {source_currency} to {destinatoon_currency} Rate")
  ans = cc_object.convert(amount, source_currency, destinatoon_currency)
  ans = round(ans, 4)
  return ans


**11. Function Calling Prompt Definition**

In [ ]:
fun_prompt = """
You have access to the following functions. If you choose to invoke any of them,
you MUST use the exact format:

[function_name(param1=value1, param2=value2, ...)]

You SHOULD NOT include any other text in the response if you are calling a function.

Available functions:

[
  {{
    "name": "get_current_datetime",
    "description": "Returns the current date and time.",
    "parameters": {{
      "type": "object",
      "properties": {{}},
      "required": []
    }}
  }},
  {{
    "name": "currency_converter",
    "description": "Converts a specified amount from one currency to another.",
    "parameters": {{
      "type": "object",
      "properties": {{
        "amount": {{
          "type": "number",
          "description": "The amount of money to convert."
        }},
        "source_currency": {{
          "type": "string",
          "description": "The currency code of the original currency (e.g., 'USD')."
        }},
        "destinatoon_currency": {{
          "type": "string",
          "description": "The currency code to convert to (e.g., 'EUR')."
        }}
      }},
      "required": ["amount", "source_currency", "destinatoon_currency"]
    }}
  }}
]
"""



**12. Execute Function Call with Gemma**

In [ ]:
user_question = "I want to convert 20000 Indian rupee to US Dollar."

full_prompt = f"{fun_prompt}\n\nNow answer the following question:\n{user_question}"

full_prompt = PROMPT_TEMPLATE.format(question=full_prompt)

output = gemma_lm.generate(full_prompt)

separator = '<end_of_turn>\n<start_of_turn>model'
parts = output.split(separator)
response_fun = parts[1].replace('<end_of_turn>', '').strip()
function_result = eval(response_fun)

In [ ]:
function_result

**13. Convert Function Output to Natural Language**

In [ ]:
followup_prompt = f"""
The function call `{response_fun}` was executed.
The result is: {function_result}

Write a natural, user-friendly response explaining this result. Also, ask a relevant follow-up question to be helpful or Suggest Related Information. Keep it short and skip any introductory phrases.

"""

followup_prompt = PROMPT_TEMPLATE.format(question=followup_prompt)
final_response = gemma_lm.generate(followup_prompt)

display_formatted_output(final_response, only_model_result=True)